In [2]:
import pandas as pd
import numpy as np

In [27]:
AML = pd.read_csv('large.csv', index_col = 'cif') 

Removing customers who are not using their account

In [28]:
AML = AML[AML['turnover'] > 0]

Adding AML country score

In [29]:
# our challenge country-id list
country = pd.read_csv('country.csv')
country.country_name = country.country_name.str.lower()
country.set_index('country_name',inplace=True, drop=True)
# downloaded ranking and AWL score for countries
country_aml = pd.read_csv('AML_risk_ranking.csv', names = ['Country', 'country_score', 'country_ranking'])
country_aml.Country =  country_aml.Country.str.lower()
country_aml.set_index('Country',inplace=True, drop=True)
# adding AML index and ranking to initial country list
country = country.join(country_aml)
# adding AML index and ranking to transaction datasource
AML = AML.join(country.set_index('country_id'), on = 'nationality')

Adding normalized and logged features

In [30]:
# adding normalized atm_features
AML['atm_withdrawal_norm'] = AML['atm_withdrawal']/AML['turnover']
AML['atm_deposit_norm'] = AML['atm_deposit']/AML['turnover']
AML['transaction_avg'] = AML['turnover']/AML['transaction_count']
#logging features
AML['turnover_log'] = np.log10(1+AML['turnover'])
AML['atm_withdrawal_log'] = np.log10(1+AML['atm_withdrawal'])
AML['atm_deposit_log'] = np.log10(1+AML['atm_deposit'])
AML['transaction_count_log'] = np.log10(1+AML['transaction_count'])
AML['distinct_counterparties_log'] = np.log10(1+AML['distinct_counterparties'])
AML['atm_withdrawal_norm_log'] = np.log10(1+AML['atm_withdrawal_norm'])
AML['atm_deposit_norm_log'] = np.log10(1+AML['atm_deposit_norm'])

Adding new categorie

In [13]:
AML['new_category'] = AML['category']
for idx, row in AML.iterrows():
    if AML.loc[idx, ('is_pep')] == 1:
        AML.loc[idx, ('new_category')] = 3

Adding new names for categories